## 기본 파이썬 

In [1]:
from datetime import date
from xlrd import open_workbook, xldate_as_tuple
from xlwt import Workbook

In [2]:
input_file = './data/sales_2013.xlsx'
output_file = './output_files/04-1 output.xls'

In [3]:
output_workbook = Workbook()
output_worksheet = output_workbook.add_sheet('set_of_worksheets')

In [4]:
my_sheets = [0, 1]
threshold = 1900.0
sales_column_index = 3

In [6]:
first_worksheet = True

with open_workbook(input_file) as workbook:
    data = []
    for sheet_index in range(workbook.nsheets):
        if sheet_index in my_sheets:
            worksheet = workbook.sheet_by_index(sheet_index)
            if first_worksheet:
                header_row = worksheet.row_values(0)
                data.append(header_row)
                first_worksheet = False
            for row_index in range(1, worksheet.nrows):
                row_list = []
                sale_amount = worksheet.cell_value(row_index, sales_column_index)
                if sale_amount > threshold:
                    for column_index in range(worksheet.ncols):
                        cell_value = worksheet.cell_value(row_index, column_index)
                        cell_type = worksheet.cell_type(row_index, column_index)
                        if cell_type == 3:
                            date_cell = xldate_as_tuple(cell_value, workbook.datemode)
                            date_cell = date(*date_cell[0:3]).strftime('%m/%d/%Y')
                            row_list.append(date_cell)
                        else:
                            row_list.append(cell_value)    
                if row_list:
                    data.append(row_list)
    for list_index, output_list in enumerate(data):
        for element_index, element in enumerate(output_list):
            output_worksheet.write(list_index, element_index, element)
output_workbook.save(output_file)

## 팬더스

In [7]:
import pandas as pd

output_file = './output_files/04-1 pandas_output.xls'

In [8]:
data_frame = pd.read_excel(input_file, sheet_name=my_sheets, index_col=None)

row_list = []

for worksheet_name, data in data_frame.items():
    row_list.append(data[data['Sale Amount'].astype(float) > threshold])
filtered_rows = pd.concat(row_list, axis=0, ignore_index=True)

In [10]:
writer = pd.ExcelWriter(output_file)

filtered_rows.to_excel(writer, sheet_name='set_of_worksheets', index=False)

writer.save()